In [ ]:
#Modules to import

#For Web Scraping
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

#For data manipulation
import pandas as pd
import numpy as np
from datetime import date

#To slow down the program when scraping
import time

In [ ]:
PATH = r'Path\chromedriver.exe'

In [ ]:
#Initial page request.
options = webdriver.ChromeOptions()
options.add_argument('javascript.enabled')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable-automation"])
options.add_argument("--incognito")
options.add_argument("--disable-site-isolation-trials")
options.add_argument('--headless')
options.add_argument('--disable-gpu')

driver = webdriver.Chrome(executable_path=PATH, options=options)
actions = ActionChains(driver)

#Initial Column headers. For now headers have to be adjust manually depending on positions. 
headers = ['WEEK','OPP','FG MADE','FG ATT','PCT','LNG','XP MADE','XP ATT','FPTS/G','FPTS', 'Name', 'Season']
stats = pd.DataFrame(columns=headers)
stats.loc[len(stats.index)] = np.zeros(len(headers))

seasons = ['2019', '2020', '2021']
#QB = 2, RB = 3, WR = 4, TE = 5, K = 6
#Currently can only run one position at a time. 
position = '6'

for season in seasons:
    url = f"https://fantasydata.com/nfl/fantasy-football-leaders?position={position}&season={season}&seasontype=1&scope=1&subscope=1&startweek=1&endweek=1&aggregatescope=1&range=1"
    driver.get(url)

    time.sleep(2)

    #Slice the first 50
    urls = [link.get_attribute('href') for link in driver.find_elements_by_xpath("//tbody/tr[@role='row']/td/a")] 

    for link in urls[:40]:
        driver.get(link)
        time.sleep(2)
        
        actions = ActionChains(driver)
        year = driver.find_element_by_xpath("(//span[@class='k-input ng-scope'])[position() = 2]")
        dropdown = driver.find_element_by_xpath("//select[@id='ddl_seasons']")
        
        driver.execute_script("arguments[0].click();", dropdown)
        actions.send_keys(season)
        time.sleep(1)
        actions.send_keys(Keys.ENTER).perform()
        
        player_headers = ['WEEK','OPP','FG MADE','FG ATT','PCT','LNG','XP MADE','XP ATT','FPTS/G','FPTS']
        player = pd.DataFrame(columns=player_headers)
        name = driver.find_element_by_xpath("//h1")
        scores = [x.text for x in driver.find_elements_by_xpath("(//div[@class='k-grid-content k-auto-scrollable'])[position() = 2]/table/tbody/tr/td")]
        rows = [scores[i:i+len(player_headers)] for i in range(0, len(scores), len(player_headers))]
        for row in rows:
            player.loc[len(player.index)] = row
        player['Name'] = name.text
        player['Season'] = season
        stats = pd.concat([stats, player], ignore_index=True)

driver.close()

In [ ]:
stats

In [ ]:
stats.to_excel(f"FilePath//{position} {season[0]}-{season[-1]}.xlsx",
             sheet_name=f'{position} Data', index=False)